In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sklearn.preprocessing
from sklearn import metrics
import acquire
import prepare

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

In [2]:
df = acquire.acquire_military()

In [3]:
df = prepare.prep_military(df)

In [4]:
df

,country,country_code,active_personnel,air_carriers,armored_vehicles,arty,attack_aircraft,avail_manpower,coastal_coverage,corvettes,...,subs,tanker_fleet,tanks,total_pop,trainers,transports,waterways,total_air_strength,total_sea_strength,total_land_strength
0,Afghanistan,AFG,137857,0,530,60,0,14611901,0,0,...,0,0,0,37466414,0,0,1200,0,0,603
1,Albania,ALB,8000,0,76,0,0,1513309,362,0,...,0,0,0,3088385,0,0,41,38,19,76
2,Algeria,ALG,130000,0,6461,995,67,20916812,998,6,...,6,5,2196,43576691,87,58,0,1046,42,9986
3,Angola,ANG,107000,0,1210,613,33,6728529,1600,0,...,0,0,320,33642646,61,30,1300,542,39,2261
4,Argentina,ARG,83000,0,800,247,6,20180574,4989,9,...,2,2,349,45864941,70,22,11000,413,28,1422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,Venezuela,VEN,137500,0,2534,164,10,14534577,2800,0,...,2,1,300,29069153,80,50,7100,478,35,3052
136,Vietnam,VTN,470000,0,4420,649,0,53450591,3444,14,...,6,0,2044,102789598,30,9,17702,360,91,7873
137,Yemen,YEM,45000,0,300,53,14,11551712,1906,2,...,0,0,200,30399243,30,8,0,292,20,603
138,Zambia,ZAM,16500,0,226,42,0,6677236,0,0,...,0,0,75,19077816,51,11,2250,186,0,393


In [5]:
df = prepare.split_military(df)

Train: 78 rows, 47 cols
Validate: 34 rows, 47 cols
Test: 28 rows, 47 cols


In [6]:
train, validate, test = df

In [7]:
train.shape

(78, 47)

In [8]:
train.head().T

,97,78,21,117,15
country,Philippines,Montenegro,Cameroon,Suriname,Bosnia and Herzegovina
country_code,PHI,MTN,CAM,SUR,BOS
active_personnel,130000,2000,40000,1800,9000
air_carriers,0,0,0,0,0
armored_vehicles,600,100,253,100,332
arty,298,12,77,0,736
attack_aircraft,27,0,0,0,0
avail_manpower,47651880,145,9698220,135245,2371365
coastal_coverage,36289,294,400,386,20
corvettes,1,0,0,0,0


In [9]:
numcols = [col for col in train.columns if train[col].dtype !='O']
numcols

['active_personnel',
 'air_carriers',
 'armored_vehicles',
 'arty',
 'attack_aircraft',
 'avail_manpower',
 'coastal_coverage',
 'corvettes',
 'defense_budget',
 'destroyers',
 'external_debt',
 'fighters_interceptors',
 'fit_for_service',
 'gold_foreign_ex',
 'frigates',
 'helos',
 'labor_force',
 'merch_marine_fleet',
 'mine_warfare',
 'navy_ships',
 'oil_consumption',
 'oil_production',
 'oil_reser',
 'paramilitary',
 'patrol_vess',
 'ports',
 'purchasing_power',
 'railway_coverage',
 'mil_age',
 'res_personnel',
 'road_cov',
 'rocket_proj',
 'shared_borders',
 'special_mission',
 'square_land_area',
 'subs',
 'tanker_fleet',
 'tanks',
 'total_pop',
 'trainers',
 'transports',
 'waterways',
 'total_air_strength',
 'total_sea_strength',
 'total_land_strength']